## Publish/Subscribe SHE msg to/from NETPIE

In [1]:
import random 
import paho.mqtt.client as mqtt
from paho.mqtt import client as mqtt_client
import json
import time
import requests

SHE Function

In [2]:
def keygen(noise, modulus):
    a_key = random.getrandbits((noise ** 2))
 
    while ((a_key % 2) != 1) and (a_key < (modulus ** (noise ** 2) - 1)):
        a_key = a_key + 1
 
    return a_key

def encrypt(noise, a_key, a_bit, modulus):
    a_mask          = random.getrandbits(noise)
    a_bit_remainder = a_bit % modulus
 
    while ((a_mask % modulus) != a_bit_remainder):
        a_mask = random.getrandbits(noise)
 
    return a_mask + (a_key * random.getrandbits(noise ** 5))

def decrypt(a_key, a_bit, modulus):
    return (a_bit % a_key) % modulus

def run_encSHE(key,noise,modulus,m1,m2):
    c1 = encrypt(noise, key, m1, modulus)
    c2 = encrypt(noise, key, m2, modulus)
    print("Ciphertext : ")
    print('c1 =',c1)
    print('c2 =', c2)
    return c1,c2

def run_decSHE(key,modulus,ciphertext_result):
    result = decrypt(key, ciphertext_result, modulus)
    print('Result = ',result)
    return result

def run_SHE(m1,m2,noise,modulus):
    key = keygen(noise, modulus)
    print("key:",key)
    c1 = encrypt(noise, key, m1, modulus)
    c2 = encrypt(noise, key, m2, modulus)
    print(f"ciphertext c1: {c1} \n ciphertext c2: {c2}")
    return c1, c2, key

## NETPIE Functions

In [3]:
def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to NETPIE MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)
    # Set Connecting Client ID
    client = mqtt_client.Client(client_id)
    client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client

def publish_shadow(client,m1,m2):
    msg_dict = {"data": {'c1':m1,'c2':m2,'multiplication':0,'additive':0}}
    msg = json.dumps(msg_dict)
    result = client.publish(topic, msg,qos=0, retain=False)
    
def subscribe(client_id,username):
    url = 'https://api.netpie.io/v2/device/shadow/data'
    basicAuthCredentials = (client_id, username)
    response = requests.get(url, auth=basicAuthCredentials)
    print(response.json())
    return response.json()

def run_NETPIE(c1,c2):
    client = connect_mqtt()
    client.loop_start() # Start loop
    publish_shadow(client,c1,c2) # publish data to @shadow/data/update
    time.sleep(4)
    client.loop_stop()
    result = subscribe(client_id,username)
    
    return result

In [4]:
mqtt_client_name = "Privacy"  # from NETPIE device
client_id = "4b2a7939-7f1f-45db-a15e-12b0542b0653" # client-ID from NETPIE device
username = "8tsxE4P4hE1LCDS9rChQiH5pLeHUCAxf" # Token from NETPIE device
password = "4kMH59#*lIDSAl6YMb)TVecA8DGh!1B2" # secret from NETPIE device
broker = "broker.netpie.io" # MQTT broker
topic = "@shadow/data/update"
topic_sub = "@shadow/data/updated"
protocol = "mqtt"
PUBLISH_PERIOD = 2000  # milliseconds
port = 1883

In [5]:
m1 = 1
m2 = 2
noise = 2
modulus = 4
c1,c2,key = run_SHE(m1,m2,noise,modulus)
result = run_NETPIE(c1,c2)
## Get the encrypted results from NETPIE
c_netpie_add = result["data"]["additive"]
c_netpie_mul =  result["data"]["multiplication"]

print(f"m_netpie_add = {c_netpie_add} \n c_netpie_mul = {c_netpie_mul}")
# Decrypt the results
m_NETPIE_add = run_decSHE(key,modulus,c_netpie_add)
m_NETPIE_mul = run_decSHE(key,modulus,c_netpie_mul)

key: 15
ciphertext c1: 44079590956 
 ciphertext c2: 19719069422
Connected to NETPIE MQTT Broker!
{'deviceid': '4b2a7939-7f1f-45db-a15e-12b0542b0653', 'data': {'additive': 63798660378, 'c1': 44079590956, 'c2': 19719069422, 'multiplication': 869208514154727300000}, 'rev': 11, 'modified': 1650908507570}
m_netpie_add = 63798660378 
 c_netpie_mul = 869208514154727300000
Result =  3
Result =  0
